# ipycanvas playbook

## History
* **25.12.21 v0.1**: First version xmas special

`ipycanvas` is an interactive canvas for Jupyter developed by Martin Renou.  It builds on `ipywidgets`.  See [here](https://ipycanvas.readthedocs.io/en/latest/index.html) for more details on `ipycanvas`.

## Installation

Install `ipycanvas` via `pip`:

`$ pip install ipycanvas`

Install the `jupyterlab` extension:

`$ jupyter labextension install @jupyter-widgets/jupyterlab-manager ipycanvas`

Check that the `ipycanvas` `jupyterlab` extension has been installed correctly - note that we are also running `jupyter-matplotlib` to support the [`solar-system` playbook](https://github.com/malminhas/solar-system/blob/main/solar-system-playbook.ipynb):

In [1]:
!jupyter labextension list

JupyterLab v3.2.5
/Users/malm/.virtualenvs/ipycanvas/share/jupyter/labextensions
        ipycanvas v0.10.2 enabled OK
        jupyter-matplotlib v0.10.4 enabled OK
        @jupyter-widgets/jupyterlab-manager v3.0.1 enabled OK (python, jupyterlab_widgets)

Other labextensions (built into JupyterLab)
   app dir: /Users/malm/.virtualenvs/ipycanvas/share/jupyter/lab



## Basic Usage

`Canvas` is an interactive widget built on [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/).  [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/) aka `jupyter-widgets` or simply `widgets`, are interactive HTML widgets for Jupyter notebooks and the `IPython` kernel.  This first very simple example constructs a blue border rect using `stroke_rect` inside a red rect created using `fill_rect`.  Note the canvas constructor.  A `Canvas` of size 800x600 will take 800x600 pixels on the screen.  The canvas origin (0,0) is top left.  x is down, y is across from left to right.

In [2]:
from ipycanvas import Canvas

canvas = Canvas(width=200, height=200)
canvas.fill_style = 'red'
canvas.stroke_style = 'blue'
canvas.fill_rect(25, 25, 100, 100)
canvas.clear_rect(45, 45, 60, 60)
canvas.stroke_rect(50, 50, 50, 50)
canvas

Canvas(height=200, width=200)

In [3]:
import numpy as np

n_particles = 100_000
x = np.array(np.random.rayleigh(250, n_particles), dtype=np.int32)
y = np.array(np.random.rayleigh(250, n_particles), dtype=np.int32)
size = np.random.randint(1, 3, n_particles)
canvas = Canvas(width=800, height=500)
canvas.fill_style = 'green'
canvas.fill_rects(x, y, size)

canvas

Canvas(width=800)

In [4]:
from ipycanvas import Path2D

canvas = Canvas(width=350, height=350)
path1 = Path2D('M80 80 A 45 45, 0, 0, 0, 125 125 L 125 80 Z')
canvas.fill_style = 'green'
canvas.fill(path1)
path2 = Path2D('M230 80 A 45 45, 0, 1, 0, 275 125 L 275 80 Z')
canvas.fill_style = 'purple'
canvas.fill(path2)
path3 = Path2D('M80 230 A 45 45, 0, 0, 1, 125 275 L 125 230 Z')
canvas.fill_style = 'red'
canvas.fill(path3)
path4 = Path2D('M230 230 A 45 45, 0, 1, 1, 275 275 L 275 230 Z')
canvas.fill_style = 'blue'
canvas.fill(path4)
canvas

Canvas(height=350, width=350)

## `RoughCanvas`

From the docs:
> `ipycanvas` provides a special canvas class that automatically gives a hand-drawn style to your drawings. 

In [5]:
from ipycanvas import RoughCanvas

canvas = RoughCanvas()
canvas.stroke_style = 'red'
canvas.fill_style = 'blue'
canvas.stroke_rect(100, 100, 100, 100)
canvas.fill_rect(50, 50, 100, 100)
canvas.stroke_style = 'purple'
canvas.fill_style = 'green'
canvas.stroke_circle(300, 300, 100)
canvas.fill_circle(350, 350, 100)
canvas.stroke_line(200, 200, 300, 300)
canvas.font = '32px serif'
canvas.fill_text('Drawing from Python is cool!', 10, 32)
canvas

RoughCanvas()

## Animation

General advice is to always use hold_canvas!  Also to try to use the **vectorized** version of the base methods as much as possible if you want to exectute them multiple times (`fill_circles`, `fill_rects` etc).  To also make use of `canvas.sleep` if you can instead of `from time import sleep` so that the entire animation is sent at once to the front-end, making a smoother animation whatever the server latency. The following is taken from the example [here](https://github.com/martinRenou/ipycanvas/blob/master/examples/animation.ipynb).

In [6]:
from math import pi, cos, sin

size = 200

def draw(canvas, t):
    step = 20
    t1 = t / 1000.0
    x = 0
    while x < size + step:
        y = 0
        while y < size + step:
            x_angle = y_angle = 2 * pi
            angle = x_angle * (x / size) + y_angle * (y / size)
            particle_x = x + 20 * cos(2 * pi * t1 + angle)
            particle_y = y + 20 * sin(2 * pi * t1 + angle)
            canvas.fill_circle(particle_x, particle_y, 6)
            y = y + step
        x = x + step
        
def fast_draw(canvas, t):
    """Same as draw, but using NumPy and the vectorized version of fill_circle: fill_circles"""
    step = 20
    t1 = t / 1000.0
    x = np.linspace(0, size, int(size / step))
    y = np.linspace(0, size, int(size / step))
    xv, yv = np.meshgrid(x, y)
    x_angle = y_angle = 2 * pi
    angle = x_angle * (xv / size) + y_angle * (yv / size)
    particle_x = xv + 20 * np.cos(2 * pi * t1 + angle)
    particle_y = yv + 20 * np.sin(2 * pi * t1 + angle)
    canvas.fill_circles(particle_x, particle_y, 6)
    
canvas = Canvas(width=size, height=size)
canvas.fill_style = "#fcba03"

The following example caches the entire animation before sending it to the front-end. This results in a slow execution (caching), but it ensure a smooth animation on the front-end whichever the context (local or remote server).  It's slow to execute but offers smooth animation.

In [7]:
from ipycanvas import hold_canvas

with hold_canvas(canvas):
    for i in range(200):
        canvas.clear()
        draw(canvas, i * 20.0)
        canvas.sleep(20)
canvas

Canvas(height=200, width=200)

This example uses `canvas.sleep` and the vectorized `fill_circles`.  It is the best approach - super fast locally, super fast on a remote server.

In [8]:
with hold_canvas(canvas):
    for i in range(200):
        canvas.clear()
        fast_draw(canvas, i * 20.0)
        canvas.sleep(20)   
canvas

Canvas(height=200, width=200)

## Fractal tree

Taken from [this notebook](https://github.com/martinRenou/ipycanvas/blob/master/examples/fractals_tree.ipynb). The button in this example is another `pywidgets` interactive control. The `canvas` commands used are as follows:
* `clear()` clears the entire canvas.
* `restore()` restores the most recently saved canvas state.
* `translate()` moves the canvas and its origin on the grid.
* `rotate()` rotates the canvas clockwise around the current origin by the angle number of radians.
* `save()` saves the entire state of the canvas.

Note the use of `hold_canvas` to capture all canvas commands.  From the `ipycanvas` documentation:
> the `hold_canvas` context manager which allows you to hold all the commands and send them in a single batch at the end. For optimal performance you should try to use hold_canvas as much as possible.

Canvas states are stored on a stack. Every time the `save()` method is called, the current drawing state is pushed onto the stack. A drawing state consists of:
* The transformations that have been applied (i.e. translate, rotate and scale – see next section).
* The current values of the following attributes: `stroke_style`, `fill_style`, `global_alpha`, `line_width`, `line_cap`, `line_join`, `miter_limit`, `line_dash_offset`, `global_composite_operation`, `font`, `text_align`, `text_baseline`, `direction`.
The `save()` method can be called as many times as you like. Each time the `restore()` method is called, the last saved state is popped off the stack and all saved settings are restored.

In [9]:
from math import pi
from random import uniform
from ipywidgets import Button

def recursive_draw_leaf(canvas, length, r_angle, r_factor, l_angle, l_factor):
    canvas.stroke_line(0, 0, 0, -length)
    canvas.translate(0, -length)
    if length > 5:
        canvas.save()
        canvas.rotate(r_angle)
        recursive_draw_leaf(
            canvas, length * r_factor, r_angle, r_factor, l_angle, l_factor
        )
        canvas.restore()
        canvas.save()
        canvas.rotate(l_angle)
        recursive_draw_leaf(
            canvas, length * l_factor, r_angle, r_factor, l_angle, l_factor
        )
        canvas.restore()
        
def draw_tree(canvas):
    with hold_canvas(canvas):
        canvas.save()
        canvas.clear()
        canvas.translate(canvas.width / 2.0, canvas.height)
        canvas.stroke_style = "black"
        r_factor = uniform(0.6, 0.8)
        l_factor = uniform(0.6, 0.8)
        r_angle = uniform(pi / 10.0, pi / 5.0)
        l_angle = uniform(-pi / 5.0, -pi / 10.0)
        recursive_draw_leaf(canvas, 150, r_angle, r_factor, l_angle, l_factor)
        canvas.restore()

def click_callback(*args, **kwargs):
    global canvas
    draw_tree(canvas)
    display(canvas)
    display(button)

canvas = Canvas(width=800, height=600)
button = Button(description="Generate tree!")
button.on_click(click_callback)
draw_tree(canvas)
display(canvas)
display(button)

Canvas(height=600, width=800)

Button(description='Generate tree!', style=ButtonStyle())